In [3]:
import math
import random

In [25]:
# sta_amount: Total contend stations number
# RA_RU: RA_RU number reserved for UORA
# Lmax: Retransmit limit
# TXOP: Transaction Opportunity (time length in ms)
# ocw_min: minimum OCW
# ocw_max: maximum OCW
def simulation(sta_amount, RA_RU, Lmax, TXOP, ocw_min, ocw_max, mode='general', Tp=10):
    counter_delay_slot=0
    success_sta=0
    # setting ocw for each transmission [ocw_min, 1st_retry_ocw...ocw_max]
    ocw = [0 for _ in range(0, Lmax)]
    ocw[0] = ocw_min
    for n in range (1, Lmax):
        temp_ocw = ocw[n-1] * 2 + 1
        if temp_ocw <= ocw_max:
            ocw[n] = temp_ocw
        else:
            ocw[n] = ocw[n-1]
    # K_n: the length of backoff interval for OCWn in term of slots.
    K = [0 for _ in range(0, Lmax) ]
    for n in range(0, Lmax):
        K = math.ceil(ocw[n] / RA_RU)
    # Imax: Reserved slot for transmission (total slots in worst case)
    Imax = 0
    Imax += (Tp - 1)
    for n in range(0, Lmax):
        Imax += math.ceil(ocw[n] / RA_RU)
    # A_i: the number of new arrivals at the beginning of the i-th slot
    A = [0 for _ in range(0, Imax) ]
    for i in range(1,Tp+1):
        A[i] = sta_amount / Tp
    # ------------------------------------ STA setting ------------------------------------------
    # set each STA's OBO counter, obo_counter = [STA1_obo_counter, STA2_obo_counter...]
    obo_counter = [random.randint(0, ocw_min) for _ in range(0, sta_amount)]
    # set STA's transmit attempt time, retrans_counter = [STA1_obo_counter, STA2_obo_counter...]
    retrans_counter = [0 for _ in range(0, sta_amount)]
    # ------------------------------------ STA setting ------------------------------------------
    # number of STAs on RA-RU
    sta_on_R = [[[] for _ in range(0,RA_RU)] for _ in range(0,Imax)]
    
    # No.i slot    
    for i in range(0, Imax):
        # No.x STA
        arrival_sta = int(min((sta_amount / Tp) * (i + 1), sta_amount))
        for x in range (0, arrival_sta):
            # decrease OBO counter by RA-RU number
            obo_counter[x] -= RA_RU
            # if counter <=0 select R
            if (obo_counter[x] <= 0 and retrans_counter[x] < 5):
                selected_R_index = random.randint(0, RA_RU-1)
                sta_on_R[i][selected_R_index].append(x)
        # check the balls in R
        for r in range(0, len(sta_on_R[i])):
            # success STA
            if (len(sta_on_R[i][r]) == 1):
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record
                counter_delay_slot += (i+1)
                success_sta += 1
            else:
                for fail_idx in sta_on_R[i][r]:
                    retrans_counter[fail_idx] += 1
                    if (retrans_counter[fail_idx] < 5):
                        obo_counter[fail_idx] = random.randint(0, ocw[retrans_counter[fail_idx]])

    success_probability = success_sta / sta_amount
    access_delay = 0
    if success_sta > 0:
        access_delay = (TXOP * counter_delay_slot) / success_sta
    
    through_put = success_sta / Imax
    utilization = success_sta / (Imax * RA_RU)
    
    return success_probability, access_delay, through_put, utilization
    


In [30]:
sample = 1000

m_list_simu = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
Ps_list_simu = [0 for i in range(20)]
Da_list_simu = [0 for i in range(20)]
T_list_simu = [0 for i in range(20)]
U_list_simu = [0 for i in range(20)]

R=9

for idx, m in enumerate(m_list_simu):
    success_probability_total = 0;
    access_delay_total = 0;
    through_put_total = 0;
    utilization_total = 0;
    for i in range(sample):
        success_probability, access_delay, through_put, utilization = simulation(sta_amount=m, RA_RU=R, Lmax=5, TXOP=5.673, ocw_min=7, ocw_max=31, mode='general')
        success_probability_total += success_probability
        access_delay_total += access_delay
        through_put_total += through_put
        utilization_total += utilization

    Ps = success_probability_total / sample
    Da = access_delay_total / sample
    T = through_put_total / sample
    U = utilization_total / sample
    
    Ps_list_simu[idx] = Ps
    Da_list_simu[idx] = Da
    T_list_simu[idx] = T
    U_list_simu[idx] = U

print(Ps_list_simu)
print(Da_list_simu)
print(T_list_simu)
print(U_list_simu)

[1.0, 0.9996999999999999, 0.9979333333333336, 0.9859250000000036, 0.9411800000000059, 0.8417, 0.7051000000000006, 0.5591625000000008, 0.4465222222222216, 0.36205999999999955, 0.29577272727272746, 0.24604166666666655, 0.20859999999999962, 0.18032142857142766, 0.15678000000000017, 0.13595625000000008, 0.12283529411764744, 0.10848888888888868, 0.09868421052631575, 0.09059499999999969]
[31.201499999999445, 32.899984588596425, 35.30328983563215, 38.995231201839566, 44.24469988181341, 49.39918746713658, 53.29157117606906, 56.28048145155135, 58.90124658436447, 61.66199837156822, 64.31092926098846, 66.90552229728355, 69.31283594002512, 71.99787421798248, 74.52961997171603, 76.74457498452068, 79.31751146860144, 81.51083725620694, 83.29701293859311, 85.20073661477984]
[0.4166666666666716, 0.8330833333333432, 1.247416666666667, 1.6432083333333456, 1.960791666666664, 2.1042499999999986, 2.0565416666666683, 1.863874999999998, 1.6744583333333356, 1.5085833333333336, 1.355625, 1.2302083333333333, 1.1